In [1]:
#!pip install autokeras
#!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc4


In [7]:
TF_MASTER = 'grpc://{}'.format('10.123.229.138')

## A Simple Example
The first step is to prepare your data. Here we use the MNIST dataset as an example


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils.data_utils import Sequence
import autokeras as ak
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.123.229.138


INFO:tensorflow:Initializing the TPU system: grpc://10.123.229.138


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [9]:
tf.random.set_seed(123)

In [10]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train, y_train), (x_test, y_test) = (x_train.astype('float32'), 
                                        y_train.astype('uint32')),( x_test.astype('float32'), 
                                                                   y_test.astype('uint32'))
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)


(60000, 28, 28)
(60000,)
[5 0 4]


The second step is to run the ImageClassifier.
It is recommended have more trials for more complicated datasets.
This is just a quick demo of MNIST, so we set max_trials to 1.
For the same reason, we set epochs to 10.
You can also leave the epochs unspecified for an adaptive number of epochs.


In [11]:
clf = ak.ImageClassifier(
      overwrite=True,
      max_trials=1,
      distribution_strategy=strategy,
      directory='gs://pytorch-tpu-nfsxgb')

In [12]:
#with strategy.scope():
#  # Initialize the image classifier.
#  clf = ak.ImageClassifier(
#      overwrite=True,
#      max_trials=1,
#      directory='gs://pytorch-tpu-nfsxgb')
    
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))


Trial 1 Complete [00h 02m 52s]
val_loss: 0.038669608533382416

Best val_loss So Far: 0.038669608533382416
Total elapsed time: 00h 02m 52s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1555 - accuracy: 0.9528
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0739 - accuracy: 0.9777
Epoch 3/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0595 - accuracy: 0.9814
Epoch 4/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0499 - accuracy: 0.9843
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0453 - accuracy: 0.9857
Epoch 6/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0404 - accuracy: 0.9866
Epoch 7/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0353 - accuracy: 0.9884
Epoch 8/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0340 - accuracy: 0.9889
Epoch 9/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0315 - accuracy: 0.9897
Epoch 10/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: gs://pytorch-tpu-nfsxgb/image_classifier/best_model/assets


INFO:tensorflow:Assets written to: gs://pytorch-tpu-nfsxgb/image_classifier/best_model/assets


[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]
  1/313 [..............................] - ETA: 4:15 - loss: 0.0022 - accuracy: 1.0000WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0013s vs `on_test_batch_end` time: 0.0125s). Check your callbacks.


313/313 [==============================] - 5s 16ms/step - loss: 0.0329 - accuracy: 0.9899
[0.03294745832681656, 0.9898999929428101]


## Reference
[ImageClassifier](/image_classifier),
[AutoModel](/auto_model/#automodel-class),
[ImageBlock](/block/#imageblock-class),
[Normalization](/block/#normalization-class),
[ImageAugmentation](/block/#image-augmentation-class),
[ResNetBlock](/block/#resnetblock-class),
[ImageInput](/node/#imageinput-class),
[ClassificationHead](/block/#classificationhead-class).
